In [ ]:
from jAER_utils.converter import aedat2numpy, DAVISaedat2numpy
import matplotlib.pyplot as plt
import os
import numpy as np
import scipy.io as sio
import cv2

In [ ]:
#data_dir = 'data/Dataset/'
data_dir='/home/getaverni/Dropbox/PhD/Projects/Gesture/Capocaccia2019_Gesture_DVS_Myo/Dataset'

In [ ]:
# this is to find the zero timestepping of the davis
def find_trigger(ts):
    return np.where(np.diff(ts) < 0)[0][0]+1

# DVS

### import .aedat and save as numpy

In [ ]:
for file in os.listdir(data_dir):
    if file.endswith(".aedat") and 'dvs' in file:
        file_path=os.path.join(data_dir, file)
        print(file.split('.')[0])
        events = aedat2numpy(file_path) #unit='ms' (by default)
        events = events[:, find_trigger(events[2]):]
        events[2] = events[2] / 1e3 #(sec.)
        print(events.shape)
        output_path=data_dir+'/'+file.split('.')[0]+'.npy'
        print(output_path)
        #np.save(output_path, events)

# DAVIS

### import .aedat

In [ ]:
output=DAVISaedat2numpy(file_path, 5000000, 'V2', 0, 'DAVIS240', 'aps', 'ms')

In [ ]:
for file in os.listdir(data_dir):
    if file.endswith(".aedat") and 'davis' in file:
        file_path=os.path.join(data_dir, file)
        print(file.split('.')[0])
        events = DAVISaedat2numpy(file_path,'dvs') #unit='ms' (by default)
        events = events[:, find_trigger(events[2]):]
        events[2] = events[2] / 1e3 #(sec.)
        print(events.shape)
        frames = DAVISaedat2numpy(file_path,'aps')
        output_path=data_dir+'/'+file.split('.')[0]+'.npy'
        print(output_path)
        np.savez(output_path, dvs_events=events, frames=frames)

### import .mat (matlab) and save as numpy
.aedat first processed in Matlab (https://github.com/inivation/AedatTools/tree/master/Matlab)

In [ ]:
sx=240
sy=180
for file in os.listdir(data_dir):
    if file.endswith(".mat") and 'davis' in file:
        file_path=os.path.join(data_dir, file)
        print(file.split('.')[0])
        output_path=data_dir+'/'+file.split('.')[0]+'.npz'
        print(output_path)
        exists = os.path.isfile(output_path)
        if exists:
            print('file {} already exists'.format(file))
            continue
        else:
            frames=[]
            frames_ts=[]
            try:
                aedat=sio.loadmat(file_path)['aedat']
                events_x=aedat['data'][0][0]['polarity'][0][0]['x'][0][0]
                events_y=aedat['data'][0][0]['polarity'][0][0]['y'][0][0]
                events_ts=aedat['data'][0][0]['polarity'][0][0]['timeStamp'][0][0]/(10**6)
                events_pol=aedat['data'][0][0]['polarity'][0][0]['polarity'][0][0]
                events_DAVIS=(np.concatenate([events_x,events_y,events_ts,events_pol], axis=1)).T
                trigger=find_trigger(events_DAVIS[2])
                events = events_DAVIS[:, trigger:]
                print(events_DAVIS.shape)
            except:
                print("No events in the file")

            ####################
            try:
                APS_samples=aedat['data'][0][0]['frame'][0][0]['samples'][0][0]
                APS_ts=aedat['data'][0][0]['frame'][0][0]['timeStampStart'][0][0]/(10**6)

                trigger_DAVIS=find_trigger(np.squeeze(np.asarray(APS_ts)))
                num_of_frames=APS_samples.shape[0]

                samples=APS_samples[trigger_DAVIS:]
                ts=APS_ts[trigger_DAVIS:]

                frames=[samples[i][0] for i in range(len(samples)) if samples[i][0].shape == (sy,sx)]#np.zeros([sy,sx,num_of_frames])
                frames_ts=[ts[i][0] for i in range(len(samples)) if samples[i][0].shape == (sy,sx)]#np.zeros([sy,sx,num_of_frames])
            except:
                print("No frame in the file")

            np.savez(output_path, dvs_events=events, frames=frames, frames_time=frames_ts)